In [1]:




# source and target list creator
import os
import random
p_path = os.path.join('C:/PHD/data/train')
dir_list = os.listdir(p_path)
print(dir_list)

class_list = ["bicycle", "bus", "car", "motorcycle","train","truck", "unk"]
path_source = "C:/PHD/data/source1_list.txt"
path_target = "C:/PHD/data/target1_list.txt"

write_source = open(path_source,"w")
write_target = open(path_target,"w")
for k, direc in enumerate(dir_list):
    if not '.txt' in direc:
        files = os.listdir(os.path.join(p_path, direc))
        for i, file in enumerate(files):
            if direc in class_list:
                class_name = direc
                file_name = os.path.join(p_path, direc, file)
                write_source.write('%s %s\n' % (file_name, class_list.index(class_name)))
            else:
                continue
p_path = os.path.join('C:/PHD/data/validation')
dir_list = os.listdir(p_path)
print(dir_list)
for k, direc in enumerate(dir_list):
    if not '.txt' in direc:
        files = os.listdir(os.path.join(p_path, direc))
        for i, file in enumerate(files):
            if direc in class_list:
                class_name = direc
            else:
                class_name = "unk"
            file_name = os.path.join(p_path, direc, file)
            write_target.write('%s %s\n' % (file_name, class_list.index(class_name)))


['aeroplane', 'bicycle', 'bus', 'car', 'horse', 'knife', 'motorcycle', 'person', 'plant', 'skateboard', 'train', 'truck']
['aeroplane', 'bicycle', 'bus', 'car', 'horse', 'knife', 'motorcycle', 'person', 'plant', 'skateboard', 'train', 'truck']


In [2]:
from __future__ import print_function
from easydict import EasyDict
import argparse
from utils.utils import *
import torch.nn as nn 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
from data_loader.get_loader import get_loader
import numpy as np
import os
from torchvision import models

# Training settings
parser = argparse.ArgumentParser(description='PyTorch Open-set DAMC')
args = parser.parse_known_args()[0]
args = EasyDict({
    "batch_size":16,
    "epochs": 1000,
    "log_interval" :100,
    "lr":0.001,
    "net":'resnet152',
    "save": True,
    "save_path":"C:/PHD/data/OPDA_BP/",
    "source_path":"C:/PHD/data/source1_list.txt",
    "target_path":"C:/PHD/data/target1_list.txt",
    "seed":1,
    "unit_size":1000,
    "update_lower": False,
    "no_cuda":False
    
})

#path_source = "C:/PHD/data/ETN/data/office/WDsource_list.txt"
#path_target = "C:/PHD/data/ETN/data/office/WDtarget_list.txt"
#"source_path":"C:/PHD/data/source_list.txt",
#    "target_path":"C:/PHD/data/target_list.txt",

args.cuda =  torch.cuda.is_available()


source_data = args.source_path
target_data = args.target_path
evaluation_data = args.target_path
batch_size = args.batch_size

data_transforms = {
    source_data: transforms.Compose([
        transforms.Scale(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    target_data: transforms.Compose([
        transforms.Scale(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    evaluation_data: transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
use_gpu = torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
train_loader, test_loader = get_loader(source_data, target_data, evaluation_data,
                                       data_transforms, batch_size=args.batch_size)
dataset_train = train_loader.load_data()
dataset_test = test_loader

num_class = 7
#class_list = ["back_pack", "bike", "bike_helmet", "bookcase", "bottle", "calculator", "desk_chair", "desk_lamp", "desktop_computer", "file_cabinet", "unk"]
class_list = ["bicycle", "bus", "car", "motorcycle", "train", "truck", "unk"]
# ['airplane', 'bicycle', 'bus', 'car', 'horse', 'knife', 'motorcycle',
# 'person', 'plant', 'skateboard', 'train', 'truck', 'unk']
    

C:\Users\tshermin\AppData\Local\Continuum\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
class GradReverse(Function):
    def __init__(self, lambd):
        self.lambd = lambd

    def forward(self, x):
        return x.view_as(x)

    def backward(self, grad_output):
        return (grad_output * -self.lambd)

def grad_reverse(x, lambd=1.0):
    return GradReverse(lambd)(x)

class ResNetBase(nn.Module):
    def __init__(self, option='resnet18', pret=True, unit_size=100):
        super(ResNetBase, self).__init__()
        self.dim = 2048
        if option == 'resnet18':
            model_ft = models.resnet18(pretrained=pret)
            self.dim = 512
        if option == 'resnet50':
            model_ft = models.resnet50(pretrained=pret)
        if option == 'resnet101':
            model_ft = models.resnet101(pretrained=pret)
        if option == 'resnet152':
            model_ft = models.resnet152(pretrained=pret)
        mod = list(model_ft.children())
        mod.pop()
        self.features = nn.Sequential(*mod)
        # default unit size 100
        self.linear1 = nn.Linear(2048, unit_size)
        self.bn1 = nn.BatchNorm1d(unit_size, affine=True)
        self.linear2 = nn.Linear(unit_size, unit_size)
        self.bn2 = nn.BatchNorm1d(unit_size, affine=True)
        self.linear3 = nn.Linear(unit_size, unit_size)
        self.bn3 = nn.BatchNorm1d(unit_size, affine=True)
        self.linear4 = nn.Linear(unit_size, unit_size)
        self.bn4 = nn.BatchNorm1d(unit_size, affine=True)
    def forward(self, x,reverse=False):

        x = self.features(x)
        x = x.view(x.size(0), self.dim)
        # best with dropout
        if reverse:
            x = F.dropout(F.relu(self.bn1(self.linear1(x))))
            x = x.detach()
            return x

        x = F.dropout(F.relu(self.bn1(self.linear1(x))), training=self.training)
        return x


class Classifier(nn.Module):
    def __init__(self, num_classes=7, unit_size=1000):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(unit_size, num_classes)
        )

    def set_lambda(self, lambd):
        self.lambd = lambd

    def forward(self, x, dropout=False, return_feat=False, reverse=False):
        if reverse:
            x = grad_reverse(x, self.lambd)
        x = self.classifier(x)
        return x
    
class GradientReverseLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, coeff, input):
        ctx.coeff = coeff
        return input.view_as(input)

    @staticmethod
    def backward(ctx, grad_outputs):
        coeff = ctx.coeff
        return None, -coeff * grad_outputs
    
class GradientReverseModule(nn.Module):
    def __init__(self, scheduler):
        super(GradientReverseModule, self).__init__()
        self.scheduler = scheduler
        self.register_buffer('global_step', torch.zeros(1))
        self.coeff = 0.0
        self.grl = GradientReverseLayer.apply

    def forward(self, x):
        self.coeff = self.scheduler(self.global_step.item())
        if self.training:
            self.global_step += 1.0
        return self.grl(self.coeff, x)
    
class LeakySoftmax(nn.Module):
    def __init__(self, coeff=1.0, dim=-1):
        super(LeakySoftmax, self).__init__()
        self.softmax = torch.nn.Softmax(dim=dim)
        self.coeff = coeff
        self.dim = dim
        
    def forward(self, x):
        shape = list(x.size())
        shape[self.dim] = 1
        leaky = (torch.ones(*shape, dtype=x.dtype) * np.log(self.coeff)).to(x.device)
        concat = torch.cat([x, leaky], dim=self.dim)
        y = self.softmax(concat)
        prob_slicing = [slice(None, None, 1) for i in shape]
        prob_slicing[self.dim] = slice(None, -1, 1)
        prob = y[tuple(prob_slicing)]
        prob_slicing[self.dim] = slice(-1, None, 1)
        total_prob = 1.0 - y[tuple(prob_slicing)]
        return prob, total_prob
    
class AugNet(nn.Module):
    def __init__(self):
        super(AugNet, self).__init__()
        classifier_output_dim = num_class-1
        self.classifier_auxiliary = nn.Sequential(
            nn.Linear(1000, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024,1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, classifier_output_dim),
            LeakySoftmax(classifier_output_dim)
        )

    def forward(self, x, k):
        y_aug, d_aug = self.classifier_auxiliary(x)
        d_aug = d_aug * k
        return y_aug, d_aug
    
def get_optimizer_visd(lr, G, C, C_aug, update_lower=False):
    if not update_lower:
        params = list(list(G.linear1.parameters()) + list(G.linear2.parameters()) + list(
            G.bn1.parameters()) + list(G.bn2.parameters())) 
    else:
        params = G.parameters()
    optimizer_g = opt.SGD(params, lr=lr, momentum=0.9, weight_decay=0.0005,nesterov=True)
    optimizer_c = opt.SGD(list(C.parameters()), momentum=0.9, lr=lr,
                          weight_decay=0.0005, nesterov=True)
    optimizer_C_aug = opt.SGD(list(C_aug.parameters()), lr=lr,momentum=0.9, weight_decay=0.0005,nesterov=True)
    #optimizer_des = opt.SGD(list(des.parameters()), lr=lr,momentum=0.9, weight_decay=0.0005,nesterov=True)
    return optimizer_g, optimizer_c, optimizer_C_aug


In [ ]:
G = ResNetBase(args.net, unit_size=args.unit_size)
C = Classifier(num_classes=num_class, unit_size=args.unit_size)
classifier_aug = AugNet()

#G.load_state_dict(torch.load('latestG3.pt'))
#C.load_state_dict(torch.load('latestC3.pt'))
#classifier_aug.load_state_dict(torch.load('latestCaug3.pt'))

if args.cuda:
    G.cuda()
    C.cuda()
    classifier_aug.cuda()
    criterion = nn.CrossEntropyLoss().cuda()
    
opt_c, opt_g, opt_caug = get_optimizer_visd(args.lr, G, C,classifier_aug,
                            update_lower=args.update_lower)

print(args.save_path)
best_acc = 0
test_acc = 0
def train(num_epoch):
    i = 0
    global best_acc
    global test_acc
    print('train start!')
    for ep in range(num_epoch):
        G.train()
        C.train()
        classifier_aug.train()
        t = 0
        for batch_idx, data in enumerate(dataset_train):
            i += 1
            if i % 1000 == 0:
                print('iteration %d', i)
            if args.cuda:
                img_s = data['S']
                label_s = data['S_label']
                img_t = data['T']
                img_s, label_s = Variable(img_s.cuda()), \
                                 Variable(label_s.cuda())
                img_t = Variable(img_t.cuda())
                label_t = data['T_label']
                label_t = Variable(label_t.cuda())
            if len(img_t) < batch_size:
                break
            if len(img_s) < batch_size:
                break
                
            opt_g.zero_grad()
            opt_c.zero_grad()
           
            feat = G(img_s)
            Caug_feat = feat.detach()
            out_s = C(feat)
            out_S = F.softmax(out_s.detach())
            probs1 = torch.sum(out_S[:, :num_class - 1], 1).view(-1, 1)
            probs2 = out_S[:, num_class - 1].contiguous().view(-1, 1)
            loss_s = criterion(out_s, label_s.long())
            loss_s.backward()
            opt_g.step()
            opt_c.step()
            
            y_onehot = torch.zeros((batch_size, num_class-1)).cuda()
            y_onehot.scatter_(1, label_s.long().unsqueeze(1), 1)
            y_onehot.requires_grad_(False)
            
            p = 1.0
            C.set_lambda(p)
            feat_t = G(img_t)
            out_t = C(feat_t)
            out_t = F.softmax(out_t)
            prob1 = torch.sum(out_t[:, :num_class - 1], 1).view(-1, 1)
            prob2 = out_t[:, num_class - 1].contiguous().view(-1, 1)
            
            Caug_feat_t = feat_t.detach()
            
            opt_caug.zero_grad()
            predictprob_source, domainprob_source = classifier_aug.forward(Caug_feat,probs1.detach())
            predictprob_target, domainprob_target = classifier_aug.forward(Caug_feat_t,prob1.detach())
            
           
            c_aug = nn.BCELoss(reduction ='none')(predictprob_source, y_onehot)
            c_aug = torch.sum(c_aug) / label_s.numel()

            adv_loss = nn.BCELoss()(domainprob_source, torch.ones_like(domainprob_source))
            adv_loss += nn.BCELoss()(domainprob_target, torch.zeros_like(domainprob_target))
            
            loss_aug = 1.0 * adv_loss + 1.0 * c_aug
            loss_aug.backward()
            opt_caug.step()
            
            opt_g.zero_grad()
            opt_c.zero_grad()
            p = 1.0
            C.set_lambda(p)
            feat_t = G(img_t)
            out_t = C(feat_t, reverse=True)
            out_t = F.softmax(out_t)
            prob1 = torch.sum(out_t[:, :num_class - 1], 1).view(-1, 1)
            prob2 = out_t[:, num_class - 1].contiguous().view(-1, 1)
            
            predictprob_target, domainprob_target = classifier_aug.forward(feat_t.detach(),prob1.detach())
        
            prob = torch.cat((prob1, prob2), 1)
            weight = (domainprob_target.detach())  
            loss_t = bce_loss(prob, weight)

            loss_t.backward()
            
            opt_g.step()
            opt_c.step()
           
    
            if batch_idx % args.log_interval == 0:
                print('Train Ep: {} [{}/{} ({:.0f}%)]\tLoss Source: {:.6f}\t Loss Target: {:.6f}\t Loss Aug: {:.6f}'.format(
                    ep, batch_idx * len(data), 70000,
                        100. * batch_idx / 70000, loss_s.data, loss_t.data, loss_aug.data))
    
            if ep > 0 and batch_idx % 1000 == 0:
                test()
                G.train()
                C.train()
                classifier_aug.train()
                
            if test_acc > best_acc:
                best_acc = test_acc
                print(best_acc)
                print('saving starts')
                torch.save(G.state_dict(),"G.pt")
                torch.save(C.state_dict(),"C.pt")
                torch.save(classifier_aug.state_dict(),"Caug.pt")
                print('saving ends')


def test():
    G.eval()
    C.eval()
    correct = 0
    size = 0
    global test_acc
    per_class_num = np.zeros((num_class))
    per_class_correct = np.zeros((num_class)).astype(np.float32)
    for batch_idx, data in enumerate(dataset_test):
        if args.cuda:
            img_t, label_t, path_t = data[0], data[1], data[2]
            img_t, label_t = Variable(img_t.cuda(), volatile=True), \
                             Variable(label_t.cuda(), volatile=True)
        feat = G(img_t)
        out_t = C(feat)
        #print(out_t.size())
        pred = out_t.data.max(1)[1]
        #print(pred)
        k = label_t.data.size()[0]
        correct += pred.eq(label_t.data.long()).cpu().sum()
        pred = pred.cpu().numpy()
        for t in range(num_class):
            t_ind = np.where(label_t.data.long().cpu().numpy() == t)
            correct_ind = np.where(pred[t_ind[0]] == t)
            per_class_correct[t] += float(len(correct_ind[0]))
            per_class_num[t] += float(len(t_ind[0]))
        size += k
    per_class_acc = per_class_correct / per_class_num
    test_acc = 100. * correct / size

    print(
        '\nTest set including unknown classes:  Accuracy: {}/{} ({:.0f}%)  ({:.4f}%)\n'.format(
            correct, size,
            100. * correct / size, float(per_class_acc.mean())))
    for ind, category in enumerate(class_list):
        print('%s:%s' % (category, per_class_acc[ind]))
        

train(args.epochs + 1)